# Secure Hash Standard: Binary Word Operations — Problem 1

## Introduction

This notebook is for the comp-theory module problems where I will document my progress .  
Problem 1 focuses on implementing a set of 32-bit word operations used in the **Secure Hash Standard (SHA-256)**.

These operations include logical functions such as `Ch`, `Maj`, and `Parity`, along with the rotation/shift functions  
`Σ0`, `Σ1`, `σ0`, and `σ1`.

All functions will be implemented in Python using **NumPy**, ensuring that all values behave as unsigned 32-bit integers  
(`np.uint32`). Each function will include:

- a clear docstring  
- a Markdown explanation  
- correct 32-bit behaviour  
- test examples verifying correctness  




## Background

The Secure Hash Standard (NIST FIPS 180-4) defines a series of operations on 32-bit words used by the SHA-256  
compression function. These operations must behave like 32-bit unsigned integers, including wraparound  
behaviour.

### Definitions (FIPS 180-4, Section 4.1.2)

Logical functions:
- **Parity(x, y, z)** = x XOR y XOR z  
- **Ch(x, y, z)** = (x AND y) XOR ((NOT x) AND z)  
- **Maj(x, y, z)** = (x AND y) XOR (x AND z) XOR (y AND z)

Big sigma functions:
- **Σ0(x)** = ROTR²(x) XOR ROTR¹³(x) XOR ROTR²²(x)  
- **Σ1(x)** = ROTR⁶(x) XOR ROTR¹¹(x) XOR ROTR²⁵(x)

Small sigma functions:
- **σ0(x)** = ROTR⁷(x) XOR ROTR¹⁸(x) XOR SHR³(x)  
- **σ1(x)** = ROTR¹⁷(x) XOR ROTR¹⁹(x) XOR SHR¹⁰(x)

Before implementing these, we create helper functions for rotation and logical right shift.


In [8]:
import numpy as np


### Helper Functions: ROTR and SHR

SHA-256 relies on two low-level bit operations:

- **Rotate Right (ROTR)** — circular rotation of x by n bits  
- **Logical Right Shift (SHR)** — shift right, filling with zeroes  

These helper functions ensure correct 32-bit wraparound behaviour.


In [9]:
def rotr(x: np.uint32, n: int) -> np.uint32:
    """
    Rotate Right (ROTR) for 32-bit words.

    Parameters
    ----------
    x : np.uint32
        The 32-bit input word.
    n : int
        Number of bits to rotate.

    Returns
    -------
    np.uint32
        The rotated 32-bit result.
    """
    x = np.uint32(x)
    return np.uint32((x >> n) | (x << (32 - n)))


def shr(x: np.uint32, n: int) -> np.uint32:
    """
    Logical Right Shift (SHR) for 32-bit words.

    Parameters
    ----------
    x : np.uint32
        The 32-bit input word.
    n : int
        Number of bits to shift.

    Returns
    -------
    np.uint32
        The shifted value (zero-filled).
    """
    return np.uint32(x >> n)


### Logical Functions: Parity, Ch, Maj

These functions operate on three 32-bit words.  
They implement core decision and mixing behaviours used by the SHA-256 compression loop.


In [10]:
def Parity(x, y, z):
    """
    Parity function for SHA-256.
    Computes x XOR y XOR z.
    """
    x, y, z = map(np.uint32, (x, y, z))
    return np.uint32(x ^ y ^ z)


def Ch(x, y, z):
    """
    Choice function.
    For each bit of x: if the bit is 1, choose y; otherwise choose z.
    """
    x, y, z = map(np.uint32, (x, y, z))
    return np.uint32((x & y) ^ (~x & z))


def Maj(x, y, z):
    """
    Majority function.
    For each bit, returns the majority value among x, y, z.
    """
    x, y, z = map(np.uint32, (x, y, z))
    return np.uint32((x & y) ^ (x & z) ^ (y & z))


### SHA-256 Sigma Functions

These operations mix input bits using rotations and shifts.  
They form the core of the SHA-256 message schedule and compression function.


In [11]:
def Sigma0(x):
    """Big Sigma 0: ROTR2 ^ ROTR13 ^ ROTR22"""
    x = np.uint32(x)
    return np.uint32(rotr(x, 2) ^ rotr(x, 13) ^ rotr(x, 22))


def Sigma1(x):
    """Big Sigma 1: ROTR6 ^ ROTR11 ^ ROTR25"""
    x = np.uint32(x)
    return np.uint32(rotr(x, 6) ^ rotr(x, 11) ^ rotr(x, 25))


def sigma0(x):
    """Small sigma 0: ROTR7 ^ ROTR18 ^ SHR3"""
    x = np.uint32(x)
    return np.uint32(rotr(x, 7) ^ rotr(x, 18) ^ shr(x, 3))


def sigma1(x):
    """Small sigma 1: ROTR17 ^ ROTR19 ^ SHR10"""
    x = np.uint32(x)
    return np.uint32(rotr(x, 17) ^ rotr(x, 19) ^ shr(x, 10))


### Testing the Functions

We now apply simple tests to verify correct behaviour of all functions.


In [12]:
x = np.uint32(0x12345678)

print("Sigma0 =", hex(Sigma0(x)))
print("Sigma1 =", hex(Sigma1(x)))
print("sigma0 =", hex(sigma0(x)))
print("sigma1 =", hex(sigma1(x)))

a = np.uint32(0xFFFFFFFF)
b = np.uint32(0x00000000)
c = np.uint32(0xAAAAAAAA)

print("Ch(a,b,c) =", hex(Ch(a,b,c)))
print("Maj(a,a,a) =", hex(Maj(a,a,a)))
print("Parity(x,x,x) =", hex(Parity(x,x,x)))


Sigma0 = 0x66146474
Sigma1 = 0x3561abda
sigma0 = 0xe7fce6ee
sigma1 = 0xa1f78649
Ch(a,b,c) = 0x0
Maj(a,a,a) = 0xffffffff
Parity(x,x,x) = 0x12345678


## Conclusion of problem one

In this problem, all SHA-256 binary word operations required by the Secure Hash Standard  
were implemented and verified. NumPy was used to ensure correct 32-bit behaviour.  
This completes Problem 1 and prepares the foundation for message scheduling and compression  
in later problems.


## Problem 2 — Fractional Parts of Cube Roots

In SHA-256, the 64 round constants (often called **K constants**) are derived from the fractional parts of the cube roots of the first 64 prime numbers (NIST FIPS 180-4, p.11).

For each of the first 64 primes \(p_i\):

1. Compute \( \sqrt[3]{p_i} \)
2. Take the fractional part only
3. Multiply by \(2^{32}\)
4. Take the floor to get a 32-bit integer
5. Display in hexadecimal

Below, I generate the primes, compute the constants using NumPy, and verify they match the published values in the Secure Hash Standard.


In [ ]:
import numpy as np

def primes(n: int) -> list[int]:
    """
    Generate the first n prime numbers using trial division.

    Parameters
    ----------
    n : int
        Number of primes to generate (n >= 0).

    Returns
    -------
    list[int]
        A list containing the first n primes in ascending order.
    """
    if n < 0:
        raise ValueError("n must be >= 0")
    if n == 0:
        return []

    out: list[int] = []
    candidate = 2

    while len(out) < n:
        is_prime = True
        for p in out:
            if p * p > candidate:
                break
            if candidate % p == 0:
                is_prime = False
                break
        if is_prime:
            out.append(candidate)
        candidate += 1 if candidate == 2 else 2  # after 2, check only odd numbers

    return out


# Quick sanity checks
assert primes(1) == [2]
assert primes(5) == [2, 3, 5, 7, 11]
assert primes(0) == []

print("First 10 primes:", primes(10))


In [ ]:
# Step 1: first 64 primes
p64 = primes(64)

# Step 2: cube roots (float64 for precision)
roots = np.cbrt(np.array(p64, dtype=np.float64))

# Step 3: fractional part
frac = roots - np.floor(roots)

# Step 4: first 32 bits of fractional part
k_values = np.floor(frac * (2**32)).astype(np.uint32)

# Step 5: display as 8-hex-digit strings
k_hex = [f"{int(v):08x}" for v in k_values]

print("First 8 computed K constants:", k_hex[:8])
print("Last 8 computed K constants:", k_hex[-8:])
print("Total constants:", len(k_hex))


In [ ]:
# Published SHA-256 K constants from NIST FIPS 180-4 (p.11)
K_FIPS = [
    "428a2f98","71374491","b5c0fbcf","e9b5dba5","3956c25b","59f111f1","923f82a4","ab1c5ed5",
    "d807aa98","12835b01","243185be","550c7dc3","72be5d74","80deb1fe","9bdc06a7","c19bf174",
    "e49b69c1","efbe4786","0fc19dc6","240ca1cc","2de92c6f","4a7484aa","5cb0a9dc","76f988da",
    "983e5152","a831c66d","b00327c8","bf597fc7","c6e00bf3","d5a79147","06ca6351","14292967",
    "27b70a85","2e1b2138","4d2c6dfc","53380d13","650a7354","766a0abb","81c2c92e","92722c85",
    "a2bfe8a1","a81a664b","c24b8b70","c76c51a3","d192e819","d6990624","f40e3585","106aa070",
    "19a4c116","1e376c08","2748774c","34b0bcb5","391c0cb3","4ed8aa4a","5b9cca4f","682e6ff3",
    "748f82ee","78a5636f","84c87814","8cc70208","90befffa","a4506ceb","bef9a3f7","c67178f2"
]

assert len(K_FIPS) == 64
assert k_hex == K_FIPS, "Computed constants do not match FIPS 180-4!"

print("✅ All 64 SHA-256 K constants match NIST FIPS 180-4 (p.11).")


### Conclusion of Problem Two

The computed constants exactly match the 64 SHA-256 round constants published in
NIST FIPS 180-4 (page 11). This confirms that the procedure of extracting the first
32 bits of the fractional parts of the cube roots of the first 64 prime numbers has
been implemented correctly.

Deriving these constants programmatically demonstrates how seemingly arbitrary
values used in cryptographic algorithms are, in fact, deterministically generated
from well-defined mathematical properties. This helps prevent the use of
“hidden” or biased constants and contributes to the transparency of the SHA-256
design.
